In [75]:
!pip install ratelimit

  Stored in directory: /root/.cache/pip/wheels/05/d9/82/3c6044cf1a54aab9151612458446d9b17a38416869e1b1d9b8
Successfully built ratelimit


In [0]:
import pandas as pd
import requests
import json
import numpy as np 
from ratelimit import limits, sleep_and_retry

In [0]:
api_key = ''

SearchPeople = pd.DataFrame.from_dict({"data":[]})
SearchMovies = pd.DataFrame.from_dict({"data":[]})

IDPeople = pd.DataFrame.from_dict({"data":[]})
IDMovies = pd.DataFrame.from_dict({"data":[]})

In [0]:
# The official limit is 
# 40 request per 10 seconds

TWO_SECOND = 2

@sleep_and_retry
@limits(calls=7, period=TWO_SECOND)
def call_api(url):
    response = requests.get(url)
    if response.status_code == 429:
        raise Exception('API response: {}'.format(response.status_code))
    return response

In [0]:
# Search a person
def searchPerson(person):
  global SearchPeople
  try:
    return SearchPeople.loc[person,"data"]
  except: 
    url = "https://api.themoviedb.org/3/search/person?api_key="+api_key+"&query=\""+person+"\""
    response = call_api(url)
    data = response.json()
    SearchPeople = SearchPeople.append(pd.Series({'data':data},name=person))
    return data
# Search a movie
def searchMovie(movie):
  global SearchMovies
  try:
    return SearchMovies.loc[movie,"data"]
  except: 
    url = "https://api.themoviedb.org/3/search/movie?api_key="+api_key+"&query=\""+movie+"\""
    response = call_api(url)
    data = response.json()
    SearchMovies = SearchMovies.append(pd.Series({'data':data},name=movie))
    return data

In [0]:
# Person and movie credits
def getPerson(person_id):
  global IDPeople
  try:
    return IDPeople.loc[person_id,"data"]
  except: 
    url = "https://api.themoviedb.org/3/person/"+str(person_id)+"?api_key="+api_key+"&append_to_response=movie_credits"
    response = call_api(url)
    data = response.json()
    IDPeople = IDPeople.append(pd.Series({'data':data},name=person_id))
    return data

# Movie and credits
def getMovie(movie_id):
  global IDMovies
  try:
    return IDMovies.loc[movie_id,"data"]
  except: 
    url = "https://api.themoviedb.org/3/movie/"+str(movie_id)+"?api_key="+api_key+"&append_to_response=credits"
    response = call_api(url)
    data = response.json()
    IDMovies = IDMovies.append(pd.Series({'data':data},name=movie_id))
    return data

In [0]:
# List of person with a job in a crew
def getJobFrom(job, crew):
  return [value for value in crew if value["job"] == job]

def getNameFrom(name, crew):
  return [value for value in crew if value["name"] == name]

# Crew of a movie
def getCrew(movie_id):
  movie = getMovie(movie_id)
  try:
    return [value for value in movie["credits"]["crew"]]
  except:
    return movie["status_message"]  
  
# Cast of a movie
def getCast(movie_id):
  movie = getMovie(movie_id)
  try:
    return [value for value in movie["credits"]["cast"]]
  except:
    return movie["status_message"]

In [0]:
crew = getPerson(searchPerson("Stan Lee")["results"][0]["id"])["movie_credits"]["crew"]
movies_ids = [movie["id"] for movie in crew]
movies_ids = np.unique(np.array(movies_ids))

In [0]:
director = searchPerson("Stan Lee")["results"][0]

In [0]:
movies = [getMovie(movie) for movie in movies_ids]

In [0]:
some_people = [(cast["name"]) for movie in movies for cast in movie["credits"]["cast"]] 

def getRoles(name, movies):
  return [(movie["title"],cast["character"]) for movie in movies for cast in movie["credits"]["cast"] if cast["name"] == name] 

def getCharacter(char, movies):
  return [(movie["title"],cast["name"], cast["character"]) for movie in movies for cast in movie["credits"]["cast"] if char in cast["character"]] 

In [220]:
getRoles("Chris Evans", movies)

[('Captain America: The First Avenger', 'Steve Rogers / Captain America'),
 ('Fantastic Four: Rise of the Silver Surfer', 'Johnny Storm / Human Torch'),
 ('Fantastic Four', 'Johnny Storm / Human Torch'),
 ('The Avengers', 'Steve Rogers / Captain America'),
 ('Thor: The Dark World', 'Loki as Captain America (uncredited)'),
 ('Avengers: Age of Ultron', 'Steve Rogers / Captain America'),
 ('Captain America: The Winter Soldier', 'Steve Rogers / Captain America'),
 ('Ant-Man', 'Steve Rogers / Captain America (uncredited)'),
 ('Captain America: Civil War', 'Steve Rogers / Captain America'),
 ('Avengers: Endgame', 'Steve Rogers / Captain America'),
 ('Avengers: Infinity War', 'Steve Rogers / Captain America'),
 ('Captain Marvel', 'Steve Rogers / Captain America (uncredited)'),
 ('Spider-Man: Homecoming', 'Steve Rogers / Captain America')]

In [218]:
pd.Series(some_people).value_counts().head(10)

Stan Lee              47
Chris Evans           13
Samuel L. Jackson     11
Hugh Jackman          11
Robert Downey Jr.     10
Fred Tatasciore        9
Chris Hemsworth        9
Gwyneth Paltrow        8
Mark Ruffalo           8
Scarlett Johansson     8
dtype: int64

In [0]:
batman_movies_ids = [movie["id"] for movie in searchMovie("Batman")["results"]]
batman_movies_ids = np.unique(np.array(batman_movies_ids))
batman_movies = [getMovie(movie) for movie in batman_movies_ids]

batman_cast = [(cast["name"]) for movie in batman_movies for cast in movie["credits"]["cast"]] 


In [236]:
getCharacter("Batman", batman_movies)

[('Batman', 'Michael Keaton', 'Bruce Wayne / Batman'),
 ('Batman Begins', 'Christian Bale', 'Bruce Wayne / Batman'),
 ('Batman Returns', 'Michael Keaton', 'Bruce Wayne / Batman'),
 ('Batman Forever', 'Val Kilmer', 'Bruce Wayne / Batman'),
 ('Batman & Robin', 'George Clooney', 'Bruce Wayne / Batman'),
 ('Batman', 'Adam West', 'Batman / Bruce Wayne'),
 ('Superman/Batman: Public Enemies',
  'Kevin Conroy',
  'Batman / Bruce Wayne (voice)'),
 ('Batman: Year One', 'Ben McKenzie', 'Batman/Bruce Wayne (voice)'),
 ('Batman', 'Lewis Wilson', 'Bruce Wayne, aka Batman'),
 ('Batman v Superman: Dawn of Justice', 'Ben Affleck', 'Bruce Wayne / Batman'),
 ('Batman: Assault on Arkham', 'Kevin Conroy', 'Batman / Bruce Wayne (voice)'),
 ('Batman vs. Robin', "Jason O'Mara", 'Bruce Wayne / Batman (voice)'),
 ('The Lego Batman Movie', 'Will Arnett', 'Batman / Bruce Wayne (voice)'),
 ('Batman: Bad Blood', "Jason O'Mara", 'Batman / Bruce Wayne (voice)'),
 ('Batman: The Killing Joke', 'Kevin Conroy', 'Bruce Wa